<a href="https://colab.research.google.com/github/anna-asmaryan/NSBE-Hacks/blob/email_phishing/Email_phishing_logistic_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

# Set the path to the files you'd like to load
file_paths = ["CEAS_08.csv", "Nazario.csv", "Nigerian_Fraud.csv", "SpamAssasin.csv"]

# Create empty dataframe with sender email, reciever, subject, body, and label
df = pd.DataFrame(columns=["sender", "receiver", "date", "subject", "body", "urls", "label"])

# Load the latest version
for file_path in file_paths:
  df_temp = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "naserabdullahalam/phishing-email-dataset",
    file_path,
    # Provide any additional arguments like
    # sql_query or pandas_kwargs. See the
    # documenation for more information:
    # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
  )
  df_temp = df_temp.loc[:, ["sender", "receiver", "date", "subject", "body", "urls", "label"]]

  df = pd.concat([df, df_temp])

print("First 5 records:", df.head())
print("Dimensions: ", df.shape[0])


#Check for missing values
import numpy as np

# Deleting rows with missing body text
df = df.dropna(subset=['body'])

#Tokenize body text
!pip install nltk

#import tokenization rsrcs
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation

nltk.download('punkt')       # For tokenization
nltk.download('stopwords')   # For stopwords
nltk.download('wordnet')     # For lemmatization

import nltk
nltk.download('punkt_tab')

# Initialize the lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Function to tokenize, remove stopwords, lemmatize, stem, and join the tokens
def tokenize_remove_stopwords_lemmatize_stem_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Get stopwords in English and punctuation
    stop_words = set(stopwords.words('english')) | set(punctuation)

    # Tokenize, remove stopwords, lemmatize, and then stem
    processed_tokens = [
        stemmer.stem(lemmatizer.lemmatize(word.lower()))  # Lemmatize and then stem
        for word in tokens
        if word.lower() not in stop_words  # Remove stopwords and punctuation
    ]

    # Join the tokens into a single string (with spaces or commas between words)
    return ' '.join(processed_tokens)  # You can change the separator if you prefer commas

# Apply the processing function to the 'body' column
df['body_processed'] = df['body'].apply(tokenize_remove_stopwords_lemmatize_stem_text)

# Now, 'body_processed' contains processed text as a string
print(df['body_processed'].head())

print(df.head())


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Step 1: Split your data into train, validation, and test sets (70% train,
# 15% validation, 15% test)
X_train, X_temp, y_train, y_temp = train_test_split(df['body_processed'],
                          df['label'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5,
                                                random_state=42)

# Step 2: Vectorize the training, validation, and test sets using TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data
X_train_vec = vectorizer.fit_transform(X_train)

# Transform the validation and test sets (don't fit again, just transform)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)



First 5 records:                                               sender  \
0                   Young Esposito <Young@iworld.de>   
1                       Mok <ipline's1983@icable.ph>   
2  Daily Top 10 <Karmandeep-opengevl@universalnet...   
3                 Michael Parker <ivqrnai@pobox.com>   
4  Gretchen Suggs <externalsep1@loanofficertool.com>   

                                         receiver  \
0                     user4@gvc.ceas-challenge.cc   
1                   user2.2@gvc.ceas-challenge.cc   
2                   user2.9@gvc.ceas-challenge.cc   
3  SpamAssassin Dev <xrh@spamassassin.apache.org>   
4                   user2.2@gvc.ceas-challenge.cc   

                              date  \
0  Tue, 05 Aug 2008 16:31:02 -0700   
1  Tue, 05 Aug 2008 18:31:03 -0500   
2  Tue, 05 Aug 2008 20:28:00 -1200   
3  Tue, 05 Aug 2008 17:31:20 -0600   
4  Tue, 05 Aug 2008 19:31:21 -0400   

                                             subject  \
0                          Never agree to 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


0    buck troubl caus small dimens soon becom lover...
1    upgrad sex pleasur techniqu http //www.brightm...
2    +=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=...
3    would anyon object remov .so list .so tld basi...
4    welcomefastshippingcustomersupport http //7iwf...
Name: body_processed, dtype: object
                                              sender  \
0                   Young Esposito <Young@iworld.de>   
1                       Mok <ipline's1983@icable.ph>   
2  Daily Top 10 <Karmandeep-opengevl@universalnet...   
3                 Michael Parker <ivqrnai@pobox.com>   
4  Gretchen Suggs <externalsep1@loanofficertool.com>   

                                         receiver  \
0                     user4@gvc.ceas-challenge.cc   
1                   user2.2@gvc.ceas-challenge.cc   
2                   user2.9@gvc.ceas-challenge.cc   
3  SpamAssassin Dev <xrh@spamassassin.apache.org>   
4                   user2.2@gvc.ceas-challenge.cc   

                              dat

In [43]:
# Before fitting the model, convert the target variable to numeric type:
y_train = pd.to_numeric(y_train, errors='coerce').astype(int)  # errors='coerce' to handle non-numeric values
y_val = pd.to_numeric(y_val, errors='coerce').astype(int)
y_test = pd.to_numeric(y_test, errors='coerce').astype(int)

# Step 3: Train Logistic Regression model on the training data
model = LogisticRegression(max_iter=1000)  # Increase max_iter if convergence
#is an issue
model.fit(X_train_vec, y_train)

# Get the training score
training_score = model.score(X_train_vec, y_train)
print(f"Training Accuracy: {training_score}")

# Step 4: Validate the model using the validation set
y_val_pred = model.predict(X_val_vec)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")



Training Accuracy: 0.9926363141457265
Validation Accuracy: 0.9860943976467442


In [44]:
df.d(types)

AttributeError: 'DataFrame' object has no attribute 'd'

In [ ]:
# Step 5: Test the model using the test set
y_test_pred = model.predict(X_test_vec)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

# Step 6: Confusion Matrix to evaluate performance
print("Confusion Matrix for Test Data:")
print(confusion_matrix(y_test, y_test_pred))